## Домашнее задание к уроку №8

1. Можно ли отобрать наиболее значимые признаки с помощью PCA? Ответ объясните.

С помощью PCA нельзя отобрать какие-либо признаки из выборки. Можно построить новые признаки на основе старых, которые будут содержать какую-то долю информации из выборки.

2. (*) Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd(). Применить к данным на уроке и сравнить ответы.

Сначала повторим реализацию PCA из урока.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
X.shape

(150, 4)

In [3]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по признаку
means = X_.mean(axis=0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for j in range(cols):
    for i in range(rows):
        X_[i, j] /= std[j]

In [4]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i[0])

Собственные значения в порядке убывания:
437.77467247979905
137.1045707202107
22.01353133569719
3.107225464292897


In [5]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [6]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_.dot(W)
Z.shape

(150, 2)

Напишем функцию, реализующую метод главных компонент через сингулярное разложение матрицы.

In [20]:
U, D, V = np.linalg.svd(X_)
U.shape, D.shape, V.shape

((150, 150), (4,), (4, 4))

In [34]:
def pca_svd(X, n):
# Функция возвращает новую матрицу "объекты-признаки" и печатает матрицу весов
    U, D, V = np.linalg.svd(X)
    assert n <= len(V[:,0]), 'Новое количество признаков больше старого'
    W = V.T[:,:2]
    print(W)
    Z = X.dot(W)
    return Z

In [37]:
(pca_svd(X_, 2)).shape

[[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


(150, 2)

Матрица весов и размер новой матрицы "объекты-признаки" совпали. Метод работает.